In [ ]:
import pandas as pd
import numpy as np
import statistics as stat
from lxml import html
import requests
from bs4 import BeautifulSoup
import csv


%matplotlib inline

In [ ]:
#Old version of base file had encoding error in one cell and couldn't be read. Downloaded new fresh version from Katharine
#df = pd.read_csv('C:/Users/sam_a/Desktop/uf-data-mining-and-analysis-master/data/lobsters_full_2017_cleaned.csv', encoding='latin-1')

In [ ]:
df = pd.read_csv('C:/Users/sam_a/Downloads/uf-data-analytics-project/lobsters_full_2017_cleaned.csv', index_col='Unnamed: 0')

In [ ]:
df.head()

In [ ]:
#add new engagement score column
df["engagement_score"] = df["upvotes"] + df["downvotes"] + df["comment_count"]

In [ ]:
df.head()

In [ ]:
#set index
#df = df.set_index('Unnamed: 0')
og_df = df

In [ ]:
#df.to_csv(r'C:\Users\Anna\Documents\Spring 2019\uf-data-mining-and-analysis-master\data\lobsters_full_2017_cleaned_FINALPROJECT.csv')

In [ ]:
df['engagement_score'].hist()

In [ ]:
#Chart engagement score by usn
total_engagement_score_by_usn = df.groupby('username')['engagement_score'].sum()
print(total_engagement_score_by_usn)
total_engagement_score_by_usn.plot()

In [ ]:
#chart number of posts by usn
total_posts_by_usn = df.groupby('username')['engagement_score'].count()
df= total_posts_by_usn
total_posts_by_usn = df.drop(df[df < 5].index)
print(total_posts_by_usn)
total_posts_by_usn.plot()

In [ ]:
#Find average post engagement per username
avg_engagement_by_user = total_engagement_score_by_usn / total_posts_by_usn
avg_engagement_by_user = avg_engagement_by_user.dropna()
print(avg_engagement_by_user)

In [ ]:
#find the standard deviation of the engagement score
stdev_avg_engagement = stat.stdev(avg_engagement_by_user)
print(stdev_avg_engagement)

In [ ]:
#mean of th engagement score
mean_avg_engagement = stat.mean(avg_engagement_by_user)
print(mean_avg_engagement)

In [ ]:
#find standard deviation levels
one_eng_stdev = stdev_avg_engagement + mean_avg_engagement
two_eng_stdev = (stdev_avg_engagement*2) + mean_avg_engagement
three_eng_stdev = (stdev_avg_engagement*3) + mean_avg_engagement

In [ ]:
df= avg_engagement_by_user

In [ ]:
stdev1df = df.drop(df[df < one_eng_stdev].index)
stdev2df = df.drop(df[df < two_eng_stdev].index)
stdev3df = df.drop(df[df < three_eng_stdev].index)

In [ ]:
print(stdev1df)
stdev1df.plot(kind= 'bar')

In [ ]:
print(stdev2df)
stdev1df.plot(kind= 'bar')

In [ ]:
print(stdev3df)
stdev1df.plot(kind= 'bar')

In [ ]:
df = og_df

In [ ]:
df.head()

In [ ]:
df = df.drop(df[df.comment_count < 1].index)

In [ ]:
df.head()

In [ ]:
comment_page = df['comments_url'].values

In [ ]:
comment_page= comment_page.tolist()

In [ ]:
#print(comment_page)

In [ ]:
comment_page = 'https://lobste.rs/s/002cgs/resilience_ruby'

In [ ]:
'''#read webpage 
for URL in comment_page:
    try:
        page = requests.get(row[2], timeout=5)
    except (requests.ConnectionError, requests.HTTPError, requests.Timeout) as e:
        logging.exception(e)
    except requests.RequestException as e:
        logging.exception(e) 
        sys.exit(1)  '''

In [ ]:
page = requests.get(comment_page, timeout=5)

In [ ]:
#brew the soup
soup = BeautifulSoup(page.text)

In [ ]:
print(soup)

In [ ]:
#find all comments on Lobste.rs post
comment_text = soup.find_all('p')

In [ ]:
print(comment_text)

In [ ]:
comment_text = str(comment_text)

In [ ]:
def remove_html_tags(text):
    """Remove html tags from a string"""
    import re
    clean = re.compile('<.*?>')
    return re.sub(clean, '', text)

In [ ]:
comment_text = remove_html_tags(comment_text)

In [ ]:
import nltk
import warnings
warnings.filterwarnings('ignore')
from nltk.sentiment.vader import SentimentIntensityAnalyzer
#nltk.download('vader_lexicon')

sia = SentimentIntensityAnalyzer()

In [ ]:
 sentiment = sia.polarity_scores(comment_text)['compound']

In [ ]:
print(sentiment)

In [ ]:
#og_df['comment_sentiment'] = sentitment
#og_df['engagement_score'].corr(og_df['comment_sentiment'])